# Imports

In [1]:
import string
import os
import re

import torch

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from IPython.display import clear_output
from random import sample
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device, "never mind, cuz will use cpu")

cpu


In [3]:
with open("/content/sequences.fasta", "r") as f:
    text = f.readlines()

print(f"{len(text)} строк")

9605

# DataPreparing

In [4]:
text[0]

'>Caenorhabditis|NP_505463.1|H2A|cH2A organism=Caenorhabditis elegans phylum=Nematoda class=Chromadorea\n'

 > последовательность гистона состоит из 17-ти строк (включая строку с идентификатором)

In [5]:
text[:18]

['>Caenorhabditis|NP_505463.1|H2A|cH2A organism=Caenorhabditis elegans phylum=Nematoda class=Chromadorea\n',
 '------------------------------------------------------------\n',
 '------------------------------------------------------------\n',
 '------------------------------------------------------------\n',
 '-MSGR-------------------------------------------------------\n',
 '----------GKG-G-----------------------------------KAK-------\n',
 '----TGG-KA---KS------------------R-SSR----AGLQFPVG----------\n',
 '------------------------------------------------------------\n',
 '-RLHRIL--RKGNYAQ--RV-----------GA-------------GAPVYL--A----A\n',
 'VLEYLAA----------EV----LELA----GN----------AA--R----D-N----K\n',
 'KT----------RI-------APRHL-QLA--------VRNDEELN-KLLA-G---VT--\n',
 '-----------IAQG-----------GVLPNIQA----------VL--LPKKTAGDKE--\n',
 '------------------------------------------------------------\n',
 '------------------------------------------------------------\n',
 '------------------

In [6]:
histon_line_length = 17

In [7]:
histons_number = str(text).count(">")
histons_number
# столько гистонов

565

In [8]:
unique_chars = set()
histone_types = set()

for line in text:
    if line.startswith('>'):
        # тип гистона
        histone_type = line.split('|')[2]
        histone_types.add(histone_type)
    else:
        # Символы из последовательности гистона
        unique_chars.update(line.strip())

# Объединяем множества
vocab = unique_chars | histone_types

vocab_size = len(vocab)
print(f"Количество элементов в словаре vocab: {vocab_size}")


Количество элементов в словаре vocab: 28


In [9]:
token2idx = {c: i for i, c in enumerate(vocab)}
idx2token = {i: c for i, c in enumerate(vocab)}

In [10]:
id = token2idx["D"]
id

27

In [11]:
idx2token[id]

'H3'

> делаем обучающую выборку:
 * Х - послед. гистона
 * у - тип гистона

In [13]:
histon_types = []
histons = []

for i in range(histons_number):
    histon_type_i = i * histon_line_length
    histon_start = histon_type_i + 1
    histon_end = (i+1) * histon_line_length

    histon_types.append(text[histon_type_i].split('|')[2])
    histons.append(str("".join(text[histon_start:histon_end])).replace('\n', ''))

histons = np.array(histons)
histon_types = np.array(histon_types)

In [14]:
histons.shape, histon_types.shape

((565,), (565,))

In [15]:
histons_tensor = torch.tensor([[ token2idx[char] for char in token ] for token in histons], dtype=int)
histon_types_tensor = torch.tensor([ token2idx[token] for token in histon_types], dtype=int)
histons_tensor.shape, histon_types_tensor.shape

(torch.Size([565, 908]), torch.Size([565]))

#Model

In [49]:
class RNNWithAttentionModel(nn.Module):
   def __init__(self, vocab_size, embedding_dim, hidden_dim, random_seed=5):
       super(RNNWithAttentionModel, self).__init__()
       torch.manual_seed(random_seed)
       torch.cuda.manual_seed(random_seed)
       torch.backends.cudnn.deterministic = True
       torch.backends.cudnn.benchmark = False
       # Create an embedding layer for the vocabulary
       self.embeddings = nn.Embedding(vocab_size, embedding_dim)
       # Create an RNN layer
       self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
       # Apply a linear transformation to get the attention scores
       self.attention = nn.Linear(hidden_dim, 1)

       self.fc = nn.Linear(hidden_dim, vocab_size)
   def forward(self, x):
       x = self.embeddings(x)
       out, _ = self.rnn(x)
       attention_out = self.attention(out).squeeze(2)
       #  Get the attention weights
       attention_weights = nn.functional.softmax(attention_out, dim=1)
       # Compute the context vector
       context = torch.bmm(attention_weights.unsqueeze(1), out).squeeze(1)

       out = self.fc(context)
       return out


In [61]:
embedding_dim = 5
hidden_dim = 9
sequence_length = histons_tensor.shape[1]
batch_size = 64

train = torch.utils.data.TensorDataset(histons_tensor, histon_types_tensor)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

model = RNNWithAttentionModel(vocab_size, embedding_dim, hidden_dim)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(histons_tensor.shape, histon_types_tensor.shape)

# Training loop
num_epochs = 100
for epoch in range(num_epochs+1):
    for X, y in train_loader:
        outputs = model(X)
        loss = criterion(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy calculation
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y).float().mean().item()

    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

torch.Size([565, 908]) torch.Size([565])
Epoch [0/100], Loss: 2.8008, Accuracy: 0.1132
Epoch [10/100], Loss: 1.3954, Accuracy: 0.3962
Epoch [20/100], Loss: 1.3226, Accuracy: 0.3962
Epoch [30/100], Loss: 1.3561, Accuracy: 0.3019
Epoch [40/100], Loss: 1.0882, Accuracy: 0.6415
Epoch [50/100], Loss: 0.8370, Accuracy: 0.6792
Epoch [60/100], Loss: 0.4452, Accuracy: 0.8113
Epoch [70/100], Loss: 0.3807, Accuracy: 0.9245
Epoch [80/100], Loss: 0.2323, Accuracy: 0.9434
Epoch [90/100], Loss: 0.2198, Accuracy: 0.9623
Epoch [100/100], Loss: 0.1198, Accuracy: 0.9811


# Final Predict (to complete task)

In [51]:
seq_fasta = '''MSGRGKQGGKARTKAKTRSSRAGLQFPVGRVHRLLRKGNYAERVGAGAPVYLAAVLEYLTAEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGKVTIAQGGVLPNIQAVLLPKKTESHHKAKGK'''

In [52]:
test_seq = torch.tensor([[ token2idx[char] for char in token ] for token in seq_fasta], dtype=int).T
test_seq[0].shape

torch.Size([130])

In [53]:
histons_tensor[0].shape

torch.Size([908])

In [54]:
model.eval()
test_predict = model(test_seq)

In [55]:
_, predicted = torch.max(test_predict, 1)

In [56]:
idx2token[predicted.item()]

'H2A'